Please Run on Jupyter Notebook

In [1]:
import re
from pprint import pprint
import pickle
import os

In [2]:
def tokenize(corpus):  
    return [token for token in corpus.split(' ') if token != '']
def _conditional_prob(s, n): 
    return round(float(s.count(f'{n[0]} {n[1]}') / s.count(n[0])), 2)
def get_conditional_prob(s, ngrams):
    conditional_prob = {}
    for ngram in ngrams:  conditional_prob[ngram] = _conditional_prob(s, ngram)
    return conditional_prob

In [3]:
def _emission_prob(s, t):  
    return float(s.count(f'{t[0]}/{t[1]}') / s.count(t[0]))
def get_emission_prob(s, tagged_corpus):
    mat = [[0 for i in range(len(tokens))] for j in range(len(tokens))]; emission_prob = {}
    for t in tagged_corpus: x = (t[1], t[0]);  emission_prob[x] = _emission_prob(s, t)
    return emission_prob
def get_n_grams(s, tokens, n=2):
    s = s.lower(); s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s); ngrams = zip(*[tokens[i:] for i in range(n)])
    return list(set([ngram for ngram in ngrams]))

In [4]:
def format_corpus(corpus):
    split = corpus.split(' ');  word_tag = [];  text = ''; tags = ''
    for s in split:
        word, tag = s.split('/');  word_tag.append((word, tag));  text = text + f'{word} ';  tags = tags + f'{tag} '
    return text, tags, list(set(word_tag))
def format_dict(prob):
    format_prob = {};  keys = prob.keys()
    for k in keys:    format_prob[k[0]] = {}
    for k in keys:    format_prob[k[0]][k[1]] = prob[k]
    return format_prob
def get_start_prob(transition_prob):
    start_prob = transition_prob['<start>']; del start_prob['<start>'];  return start_prob

In [10]:
with open('corpus1.txt', 'r') as f:    all_lines = f.readlines()
corpus_tag = ' '.join([x.replace('\n', '') for x in all_lines])
tagged_corpus = [];  text = [];  tags = []
for line in all_lines:
    line = line.replace('\n', ''); txt, tag, c = format_corpus(line);txt = txt[:len(txt)-1]; tag = tag[:len(tag)-1]
    tag = f'<start> {tag} </end>'; text.append(txt);  tags.append(tag)
    for tup in c:   tagged_corpus.append(tup)
tagged_corpus = list(set(tagged_corpus)) # removing duplicates
n_grams = []; tokens = []
for t in tags:
    tkns = tokenize(t)
    for t in tkns:  tokens.append(t)
    n_gram = get_n_grams(t, tokens)
    for n in n_gram:  n_grams.append(n)
n_grams = list(set(n_grams))
for t in set(tokens):  n_grams.append((t, t))
tokens = list(set(tokens)); words = list(set(' '.join(text).split(' ')))
tag_transition_matrix = get_conditional_prob(' '.join(tags), n_grams)
emission_prob = get_emission_prob(corpus_tag, tagged_corpus)
transition_prob = format_dict(tag_transition_matrix)
emission_prob = format_dict(emission_prob); start_prob = get_start_prob(transition_prob)
hmm_model = {'Q': tokens,'A': transition_prob,'O': words,'B': emission_prob,'q0': '<start>','qF': '</end>','start_prob': start_prob}
if not os.path.isfile('hmm1.pickle'):  
    with open('hmm1.pickle', 'wb') as f:pickle.dump(hmm_model, f)
    with open('hmm1.pickle', 'rb') as f: hmm = pickle.load(f)
pprint(hmm_model)


{'A': {'</end>': {'</end>': 0.0, '<start>': 0.5},
       '<start>': {'dog': 1.0},
       'cat': {'</end>': 0.5, 'cat': 0.5},
       'dog': {'</end>': 0.25, 'cat': 0.25, 'dog': 0.25}},
 'B': {'cat': {'meow': 0.5, 'woof': 0.25}, 'dog': {'meow': 0.5, 'woof': 0.75}},
 'O': ['woof', 'meow'],
 'Q': ['cat', 'dog', '<start>', '</end>'],
 'q0': '<start>',
 'qF': '</end>',
 'start_prob': {'dog': 1.0}}


In [11]:
def print_dptable(V):
    s = "    " + " ".join(("%7d" % i) for i in range(len(V))) + "\n"
    for y in V[0]:  s += "%.7s: " % y;  s += " ".join("%4s" % ("%f" % v[y]) for v in V); s += "\n"
    print(s)
def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}];  path = {}
    for y in states:
        V[0][y] = start_p[y] * emit_p[y][obs[0]];   path[y] = [y]
    for t in range(1, len(obs)):
        V.append({});  newpath = {}
        for y in states:
            (prob, state) = max((V[t-1][y0] * trans_p[y0][y] * emit_p[y][obs[t]], y0) for y0 in states)
            V[t][y] = prob
            newpath[y] = path[state] + [y]
        path = newpath
    print_dptable(V);   (prob, state) = max((V[t][y], y) for y in states)
    return (prob, path[state])

In [12]:
with open('hmm1.pickle', 'rb') as f:     hmm = pickle.load(f)
states = tuple(hmm['Q']); observations = tuple(hmm['O']); start_probability = hmm['start_prob']
transition_probability = hmm['A']
for s in states:
    val_keys = transition_probability[s].keys()
    for s1 in states:
        if s1 not in val_keys:
            transition_probability[s][s1] = 0.0
emission_probability = hmm['B']
for s in states:
    if s not in emission_probability:
        emission_probability[s] = {}
        for o in observations:  emission_probability[s][o] = 0.0
for s in states:
    val_keys = emission_probability[s].keys()
    for o in observations:
        if o not in val_keys:     emission_probability[s][o] = 0.0
print(viterbi(observations,states,start_probability,transition_probability,emission_probability))

          0       1
<start>: 0.000000 0.000000
</end>: 0.000000 0.000000
cat: 0.000000 0.093750
dog: 0.750000 0.093750

(0.09375, ['dog', 'dog'])
